In [186]:
import pandas as pd
import wget
import numpy as np
import platform
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score, mean_squared_error

In [187]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
wget.download(url)


'course_lead_scoring.csv'

In [188]:
df = pd.read_csv('course_lead_scoring.csv')

In [189]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [190]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [191]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [192]:
cat_features = list(df.dtypes[df.dtypes == 'object'].index)

In [193]:
num_features = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [194]:
cat_features

['lead_source', 'industry', 'employment_status', 'location']

In [195]:
num_features

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [196]:
df[cat_features] = df[cat_features].fillna('NA')

In [197]:
df[num_features] = df[num_features].fillna(0)

In [198]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [199]:
df['industry'].mode()[0]

'retail'

Q1: What is the most frequent observation (mode) for the column industry? - retail

Question 2 <br>
Create the correlation matrix for the numerical features of your dataset <br>
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset <br>
What are the two features that have the biggest correlation in this dataset?

In [200]:
df_num = df.copy()

In [201]:
df_num = df[num_features]

In [202]:
df_num.describe()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
count,1462.000000,1462.000000,1462.000000,1462.000000
mean,2.031464,52472.172367,2.976744,0.506108
std,1.449717,24254.347030,1.681564,0.288465
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,44097.250000,2.000000,0.262500
50%,2.000000,57449.500000,3.000000,0.510000
75%,3.000000,68241.000000,4.000000,0.750000
max,9.000000,109899.000000,11.000000,1.000000


In [203]:
df_num.corr().unstack().sort_values(ascending = False)

number_of_courses_viewed  number_of_courses_viewed    1.000000
annual_income             annual_income               1.000000
interaction_count         interaction_count           1.000000
lead_score                lead_score                  1.000000
annual_income             interaction_count           0.027036
interaction_count         annual_income               0.027036
annual_income             lead_score                  0.015610
lead_score                annual_income               0.015610
interaction_count         lead_score                  0.009888
lead_score                interaction_count           0.009888
number_of_courses_viewed  annual_income               0.009770
annual_income             number_of_courses_viewed    0.009770
number_of_courses_viewed  lead_score                 -0.004879
lead_score                number_of_courses_viewed   -0.004879
number_of_courses_viewed  interaction_count          -0.023565
interaction_count         number_of_courses_viewed   -0

Q2: annual income and interaction count

Split data

In [204]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [205]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [206]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [207]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [208]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [209]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

Q3 <br>
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only. <br>
Round the scores to 2 decimals using round(score, 2).

In [210]:
cat_features

['lead_source', 'industry', 'employment_status', 'location']

In [211]:
def calculate_mi(series):
    return mutual_info_score(series, df_train.converted)

df_mi = df_train[cat_features].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

df_mi

,MI
lead_source,0.035396
employment_status,0.012938
industry,0.011575
location,0.004464


In [212]:
df_mi['MI'].max()

0.03539624379726594

Q3: lead_source has the biggest mutual information score

In [213]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [214]:
assert 'converted' not in df_train.columns
assert 'converted' not in df_val.columns
assert 'converted' not in df_test.columns

In [215]:
df_full_train[num_features].corrwith(df_full_train.converted).to_frame('correlation')

,correlation
number_of_courses_viewed,0.442068
annual_income,0.029612
interaction_count,0.378482
lead_score,0.225641


In [216]:
df_full_train.groupby(by='converted')[num_features].mean()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
converted,,,,
0,1.209150,51220.976035,2.137255,0.423529
1,2.528169,52691.160563,3.440845,0.557155


Q4

Now let's train a logistic regression. <br>
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset. <br>
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters: <br>
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42) <br>
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.<br>

In [217]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [218]:
cat_features

['lead_source', 'industry', 'employment_status', 'location']

In [219]:
num_features

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [220]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

In [221]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [222]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict(X_val)

In [223]:
#y_pred = model.predict_proba(X_val)[:, 1]
#converted = y_pred > 0.5
#np.round((y_val == converted).mean(),2)

In [224]:
accuracy = np.round(accuracy_score(y_val, y_pred),2)

In [225]:
accuracy

0.7

In [226]:
#accuracy = (y_val == converted).mean()

Q4: 0.7

Question 5 <br>
Let's find the least useful feature using the feature elimination technique. <br>
Train a model using the same features and parameters as in Q4 (without rounding).<br>
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.<br>
For each feature, calculate the difference between the original accuracy and the accuracy without the feature. <br>
Which of following feature has the smallest difference? <br>

'industry' <br>
'employment_status'<br>
'lead_score'<br>

In [227]:
df_train.columns.to_list()

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score']

In [228]:
df_train.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
dtype: object

In [229]:
cat_features

['lead_source', 'industry', 'employment_status', 'location']

In [230]:
features =df_train.columns.to_list()

In [231]:
original_score = accuracy

In [232]:
scores = pd.DataFrame(columns=['feature', 'accuracy', 'difference'])
for feature in features:
    subset = features.copy()
    subset.remove(feature)
    
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[subset].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    model = LogisticRegression(solver='liblinear', max_iter=1000, C=1.0, random_state=42)
    model.fit(X_train, y_train)
    
    val_dict = df_val[subset].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    scores.loc[len(scores)] = [feature, score, original_score - score]

In [233]:
scores

,feature,accuracy,difference
0,lead_source,0.703072,-0.003072
1,industry,0.699659,0.000341
2,number_of_courses_viewed,0.556314,0.143686
3,annual_income,0.853242,-0.153242
4,employment_status,0.696246,0.003754
5,location,0.709898,-0.009898
6,interaction_count,0.556314,0.143686
7,lead_score,0.706485,-0.006485


In [234]:
min_diff =scores[scores['feature'].isin(['industry','employment_status','lead_score'])].difference.min()
scores[scores.difference == min_diff]

,feature,accuracy,difference
7,lead_score,0.706485,-0.006485


Q5: Lead score

Question 6 <br>
Now let's train a regularized logistic regression. <br>
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].<br>
Train models using all the features as in Q4.<br>
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.<br>

In [235]:
features = df_train.columns.to_list()
features

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score']

In [237]:
scores_q6 = {}
C_values = [0.01, 0.1, 1, 10, 100]

for C in C_values:
    subset = features.copy()
    subset.remove(feature)

    dv = DictVectorizer(sparse=False)
    train_dict = df_train[subset].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    model = LogisticRegression(solver='liblinear', max_iter=1000, C=C, random_state=42)
    model.fit(X_train, y_train)

    val_dict = df_val[subset].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    y_pred = model.predict_proba(X_val)[:, 1]
    converted = y_pred > 0.5
    score = (y_val == converted).mean()
    scores_q6[C] = round(score, 3)
    print(f'C = {C}:\t score = {score}')

C = 0.01:	 score = 0.6962457337883959
C = 0.1:	 score = 0.6996587030716723
C = 1:	 score = 0.7064846416382252
C = 10:	 score = 0.7064846416382252
C = 100:	 score = 0.7064846416382252
